In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from scipy import linalg
from sklearn.preprocessing import normalize
from pathlib import Path
from scipy.optimize import minimize, differential_evolution



In [4]:
mpl.use('Agg')
num_modes = 10
# years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
years = ['2017']
for year in years:
    print(year)
    density_df = pd.read_csv('Data/{}_HASDM_500-575KM.txt'.format(year), delim_whitespace=True,
                                  header=None)
    
    density_np = pd.DataFrame.to_numpy(density_df)
    
    del density_df
    nt = 19
    nphi = 24

    t = np.linspace(-np.pi / 2, np.pi / 2, nt)
    phi = np.linspace(0, np.deg2rad(345), nphi)

    for i in range(density_np.shape[0]):
        if density_np[i, 7] > 1e-6:
            density_np[i, 7] = 1e-13

    max_rho = np.max(density_np[:, 7])
    density_np[:, 7] = density_np[:, 7] / max_rho

    rho_list = []
    rho_list1 = []
    rho_list2 = []

    for i in range(int(1331520 / (nt * nphi))):  
        rho_i = density_np[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 7]
        rho_polar_i = np.reshape(rho_i, (nt, nphi, 4))

        rho_list1.append(rho_polar_i)

    rho = np.array(rho_list1)

    rho_zeros = np.zeros((2920, 20, 24, 4)) 
    rho_zeros[:, :nt, :nphi, :] = rho
    del rho_list1, rho  

    training_data = rho_zeros[:2000]
    validation_data = rho_zeros[2000:]

    training_data_resh = np.reshape(training_data, newshape=(2000, 20*24*4))
    nPoints_val = len(rho_zeros) - len(training_data)
    validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))

    nPoints_val = 920  
    validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))
    rhoavg = np.mean(validation_data_resh, axis=0)  # Compute mean
    rho_msub_val = validation_data_resh.T - np.tile(rhoavg, (nPoints_val, 1)).T  # Mean-subtracted data

    rhoavg = np.mean(training_data_resh, axis=0)  # Compute mean
    nPoints = 2000
    rho_msub = training_data_resh.T - np.tile(rhoavg, (nPoints, 1)).T  # Mean-subtracted data
    pca = PCA(n_components=num_modes)
    X_pca_lin = pca.fit_transform(rho_msub_val.T)
    X_back_lin = pca.inverse_transform(X_pca_lin)
    X_back_lin = X_back_lin.T
    error_lin = rho_msub_val-X_back_lin
    error_norm_lin = linalg.norm(error_lin)
    print("PCA error:", error_norm_lin)
    x = [0.2575, 4.66323895e-12] # 3.27369962e-01 4.66323895e-14
    kpca1 = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
    # kpca1.fit(rho_msub.T)
    X_pca = kpca1.fit_transform(rho_msub_val.T)
    X_back = kpca1.inverse_transform(X_pca)
    X_back = X_back.T
    error_adv = rho_msub_val-X_back
    error_norm_adv = linalg.norm(error_adv)
    print("KPCA error:", error_norm_adv)



2017
PCA error: 0.6300055630518334
KPCA error: 0.36782246809376046


In [10]:
error_mc = np.array([[2011,2012,2013,2014,2015,2016, 2017, 2018,2019],
[1.0899209543968615, 0.847877343254818, 1.2293, 0.47959115485669795, 0.40381545672734986, 0.44245526950139896,0.6300055630518334, 0.6731482012237523, 0.8940157050228068], # pca
[0.7187770900147014, 0.664344046318804, 0.2862, 0.17178547690063284, 0.23388559484001975, 0.21349350295642938,0.36782246809376046, 0.0291577032882008, 0.2311992504638841], # kpca
[35.086653933958395, 27.611372228956483, 21.25, 16.660444098166774, 16.90782952500241, 16.808624366991978, 15.130168268574135, 53.72384988695899, 15.08172322609553]]) # autoencoder
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Density data")
plt.ylabel("Reconstruction error")
plt.semilogy(error_mc[0,:], error_mc[1,:], linewidth=2, label='PCA')
plt.semilogy(error_mc[0,:], error_mc[2,:], linewidth=2, label='KPCA')
plt.semilogy(error_mc[0,:], error_mc[3,:], linewidth=2, label='Autoencoder')

plt.legend(loc=1)
plt.grid(True)
plt.tight_layout()
plt.savefig('output/MonteCarlo.png')

In [6]:
lorenz0_tr = pd.read_csv('output/training_data0.txt')
lorenz0_vl = pd.read_csv('output/validation_data0.txt')

lorenz0_np_tr = pd.DataFrame.to_numpy(lorenz0_tr)
lorenz0_np_vl = pd.DataFrame.to_numpy(lorenz0_vl)


del lorenz0_tr, lorenz0_vl
decoded0_df = pd.read_csv('output/decoded0.txt')
decoded0_np = pd.DataFrame.to_numpy(decoded0_df)
del decoded0_df

np.random.shuffle(lorenz0_np_tr)
aux = np.concatenate((lorenz0_np_vl, decoded0_np), axis=1)
np.random.shuffle(aux)
lorenz0_np_vl = aux[:,:128]
decoded0_np = aux[:,128:]
del aux
def mykpca(x):
    try: 
        kpca_lor = KernelPCA(n_components=3, kernel="rbf", fit_inverse_transform=True, gamma=x[0],
                     alpha=x[1])  #  1e-11
        X_kpca_lor = kpca_lor.fit_transform(lorenz0_np_vl)
        X_back_kpca_lor = kpca_lor.inverse_transform(X_kpca_lor)
        error_adv = lorenz0_np_vl-X_back_kpca_lor
        error_norm_adv = linalg.norm(error_adv)
        return error_norm_adv
    except:
        return 1e+10
gamma_init = np.linspace(0.01,0.6,num=5)
alpha_init = np.linspace(1e-9, 1e-5, num=5)
res_x = []
res_value = []
for i in range(5):
    x0 = [gamma_init[i], alpha_init[i]] 
    # bounds = [(0.2, 0.4), (1e-14, 1e-12)]
    res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-14, options={'maxiter':50, 'disp': True})
    # res = differential_evolution(mykpca, bounds, maxiter=10, tol=1e-14, disp=True)
    res_x.append(res.x)
    res_value.append(res.fun)
print(res_x)
print(res_value)


error_auto_lor = lorenz0_np_vl - decoded0_np
error_norm_auto_lor = linalg.norm(error_auto_lor)
print('error_norm_autoencoder_lorenz:', error_norm_auto_lor)

pca = PCA(n_components=3)
X_pca_lin_lor = pca.fit_transform(lorenz0_np_vl)
X_back_lin_lor = pca.inverse_transform(X_pca_lin_lor)
error_adv = lorenz0_np_vl-X_back_lin_lor
error_norm_adv = linalg.norm(error_adv)
print('error_norm_lorenz:', error_norm_adv)
x = [6.88349557e-03, 1.81877613e-12]
kpca_lor = KernelPCA(n_components=3, kernel="rbf", fit_inverse_transform=True, gamma=x[0],
                     alpha=x[1])  #  0.011, 1e-11
X_kpca_lor = kpca_lor.fit_transform(lorenz0_np_vl)
X_back_kpca_lor = kpca_lor.inverse_transform(X_kpca_lor)
error_adv = lorenz0_np_vl-X_back_kpca_lor
error_norm_adv = linalg.norm(error_adv)
print('error_norm_lorenz_kpca:', error_norm_adv)


The most suitable Kernel: 

$K(X_1, X_2) = e^{(-\gamma ||X_1 - X_2||^2)}$

In [5]:
def mykpca(x):
    try: 
        kpca = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
        X_pca = kpca.fit_transform(rho_msub.T)
        X_back = kpca.inverse_transform(X_pca)
        X_back = X_back.T
        error_adv = rho_msub-X_back
        error_norm_adv = linalg.norm(error_adv)
        return error_norm_adv
    except:
        return 1e+10
gamma_init = np.linspace(0.1,0.6,num=5)
alpha_init = np.linspace(9.52148437e-14, 9.52148437e-9, num=5)
res_x = []
res_value = []
for i in range(5):
    x0 = [gamma_init[i], 4.07854175e-14] # alpha_init[i]
    # bounds = [(0.2, 0.4), (1e-14, 1e-12)]
    res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-14, options={'maxiter':50, 'disp': True})
    # res = differential_evolution(mykpca, bounds, maxiter=10, tol=1e-14, disp=True)
    res_x.append(res.x)
    res_value.append(res.fun)

asd


In [30]:
x = np.arange(-7,2, 0.25)
x = 10 ** x
# x = np.array([1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 2, 5, 10])
kpca_error = np.zeros((1, x.shape[0]))
for i in range(x.shape[0]):
    kpca1 = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[i], alpha=4.66323895e-12)
    X_pca = kpca1.fit_transform(rho_msub.T)
    X_back = kpca1.inverse_transform(X_pca)
    X_back = X_back.T
    error_adv = rho_msub-X_back
    error_norm_adv = linalg.norm(error_adv)
    # print(error_norm_adv)
    kpca_error[:, i] = error_norm_adv
    del kpca1
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Kernel parameter $\gamma$")
plt.ylabel("$L^2$ norm of reconstruction error")
plt.semilogx(x, kpca_error.T, linewidth=2)
# plt.plot(kpca_error, linewidth=2) # , label="PCA error"
# plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/kpca_vs_gamma.png')

# error_adv = np.reshape(error_adv.T, newshape=(2000, 20, 24, 4))
# plt.figure()
# plt.rcParams.update({'font.size': 14})  # increase the font size
# mpl.rcParams['legend.fontsize'] = 15
# plt.xlabel("Longitude [deg]")
# plt.ylabel("Latitude [deg]")
# plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
#              cmap="inferno", levels=900)
# plt.colorbar()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('output/ReconstructionError_kpca_adv.png')

In [5]:
pca = PCA(n_components=num_modes)
pca.fit(rho_msub.T)
X_pca_lin = pca.fit_transform(rho_msub_val.T)
X_back_lin = pca.inverse_transform(X_pca_lin)
X_back_lin = X_back_lin.T
error_lin = rho_msub_val-X_back_lin
error_norm_lin = linalg.norm(error_lin)
print(error_norm_lin)
x = [0.2575, 4.66323895e-12] # 3.27369962e-01 4.66323895e-14
kpca1 = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
kpca1.fit(rho_msub.T)
X_pca = kpca1.fit_transform(rho_msub_val.T)
X_back = kpca1.inverse_transform(X_pca)
X_back = X_back.T
error_adv = rho_msub_val-X_back
error_norm_adv = linalg.norm(error_adv)
print(error_norm_adv)

0.6185030848776232
0.41124215511636236


In [20]:
x = [3.27369962e-01, 4.66323895e-12] # 4.66323895e-14
# num_modes = 1
modes = []
PCA_error = []
KPCA_error = []

for num_modes in range(2,15):
    modes.append(num_modes)
    kpca_adv = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])

    # fftout = fftpack.fftn(rho_msub.T)

    pca = PCA(n_components=num_modes)

    # pca_fft = PCA(n_components=num_modes)
    # pca.fit(rho_msub.T)
    # X_pca_lin = pca.fit_transform(rho_msub_val.T)
    X_pca_lin = pca.fit_transform(rho_msub.T)

    # X_pca_lin_fft = pca_fft.fit_transform(fftout.real)

    # gram = cosine_kernel(rho_msub.T, rho_msub.T)
    # X_pca_man = pcam.fit_transform(gram)
    # X_pca = kpca.fit_transform(rho_msub.T)
    # explained_variance_kpca = (kpca.eigenvalues_ ** 2) / (len(rho_msub.T) - 1)

    # print(np.sum(explained_variance_kpca))

    # kpca_test = KernelPCA(n_components=1920, kernel="rbf", fit_inverse_transform=True, gamma=0.11)  #  0.11

    # kpca_test.fit(rho_msub.T)
    # X_pca_test = kpca_test.fit_transform(rho_msub_val.T)
    # X_pca_test = kpca_test.fit_transform(rho_msub.T)
    # explained_variance_kpca_test = (kpca_test.eigenvalues_ ** 2) / (len(rho_msub.T) - 1)

    # print(np.sum(explained_variance_kpca)/np.sum(explained_variance_kpca_test))

    # kpca_adv.fit(rho_msub.T)
    # X_pca_adv = kpca_adv.fit_transform(rho_msub_val.T)
    X_pca_adv = kpca_adv.fit_transform(rho_msub.T)

    X_back_adv = kpca_adv.inverse_transform(X_pca_adv)

    X_back_lin = pca.inverse_transform(X_pca_lin)
    # X_back_lin_fft = pca_fft.inverse_transform(X_pca_lin_fft)
    # X_back_lin_fft_inv = fftpack.ifftn(X_back_lin_fft)

    # K = my_kernel(X_pca,X_pca)
    # K.flat[:: nPoints + 1] += 1
    # dual_coef_ = linalg.solve(K, rho_msub.T, sym_pos=True, overwrite_a=True)
    # K = my_kernel(X_pca, X_pca)
    # X_back_man_nonl = np.dot(K, dual_coef_).T

    X_back_lin = X_back_lin.T

    # X_back = X_back.T
    X_back_adv = X_back_adv.T

    # X_back_lin_fft_inv = X_back_lin_fft_inv.T

    # error = rho_msub-X_back
    # error_norm = linalg.norm(error)
    # print('error_norm_rbf:', error_norm)  # Error in reconstruction using built-in rbf kpca

    error_adv = rho_msub-X_back_adv
    error_norm_adv = linalg.norm(error_adv)
    # print('error_norm_rbf with input alpha:', error_norm_adv)  # Error in reconstruction using built-in rbf kpca
    KPCA_error.append(error_norm_adv) 


    error_lin = rho_msub-X_back_lin
    error_norm_lin = linalg.norm(error_lin)  # , ord=inf
    # print('error_norm_lin:', error_norm_lin)   # Error in reconstruction using built-in linear pca with
    PCA_error.append(error_norm_lin) 

    # error_lin_fft = fftout.real - X_back_lin_fft.T
    # error_lin_fft = rho_msub - X_back_lin_fft_inv
    # error_norm_lin_fft = linalg.norm(error_lin_fft)  # , ord=inf
    # print('error_norm_lin_fft:', error_norm_lin_fft)  # Error in reconstruction using built-in linear pca with


    # fftin_resh = fftout.real.reshape(2000, 20, 24, 4)
    # fftout_resh = error_lin_fft.reshape(2000, 20, 24, 4)
    # error = rho_msub - X_back_lin_fft_inv
    # error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
    # Path("./output/").mkdir(parents=True, exist_ok=True)
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/fft_error.png')

    # error = rho_msub-X_back_lin
    # error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/ReconstructionError_pca.png')

    # error_adv = rho_msub-X_back_adv
    # error_adv = np.reshape(error_adv.T, newshape=(2000, 20, 24, 4))
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/ReconstructionError_kpca_adv.png')

    # plt.close('all')

data = {'number of modes':modes,'PCA error':PCA_error, 'KPCA error':KPCA_error}
df = pd.DataFrame(data)
print(df)

plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("# of modes")
plt.ylabel("Reconstruction error")
plt.plot(PCA_error, label="PCA error", linewidth=2)
plt.plot(KPCA_error, label="KPCA error", linewidth=2)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/PCA_vs_KPCA_error.png')
plt.close('all')




    number of modes  PCA error  KPCA error
0                 2  16.714446   27.560622
1                 3  11.601645   11.954918
2                 4   9.173239    8.782353
3                 5   7.596694    6.668534
4                 6   6.108573    4.950681
5                 7   4.723315    3.676183
6                 8   3.120358    2.441859
7                 9   1.551486    1.339403
8                10   1.229338    0.549276
9                11   1.019850    0.123205
10               12   0.848673    0.072738
11               13   0.717967    0.043660
12               14   0.612778    0.029549


In [9]:
for i in range(num_modes):
    plt.figure()
    plt.rcParams.update({'font.size': 14})  # increase the font size
    mpl.rcParams['legend.fontsize'] = 15
    plt.xlabel("Longitude [deg]")
    plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), training_data[10, :19, :, 0] * max_rho, cmap="viridis", levels=900)
    # X_pca X_pca_adv
    plt.plot(X_pca_adv[:, i])
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('output/KPCA_mode%d.png'%i)
plt.close('all')

In [11]:
errorae = [66.8464, 28.1242, 25.1675, 26.8981, 21.2226, 18.3057]
modes = [2, 4, 6, 8, 10, 12]
PCA_error = [16.714446, 9.173239, 6.108573, 3.120358, 1.229338, 0.848673]
KPCA_error = [25.821876, 8.507736, 4.921118, 2.429778, 0.286163, 0.072301]
datam = {'number of modes':modes,'PCA error':PCA_error, 'KPCA error':KPCA_error, 'Autoencoder erros':errorae}
df = pd.DataFrame(datam)
print(df)
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Number of modes")
plt.ylabel("Reconstruction error")
plt.plot(modes, PCA_error, label="PCA", linewidth=2)
plt.plot(modes, KPCA_error, label="KPCA", linewidth=2)
plt.plot(modes, errorae, label="Autoencoder", linewidth=2)
plt.legend()
plt.grid(True)
plt.yscale('log')

plt.tight_layout()
plt.savefig('output/error_comparison.png')
plt.close('all')

   number of modes  PCA error  KPCA error  Autoencoder erros
0                2  16.714446   25.821876            66.8464
1                4   9.173239    8.507736            28.1242
2                6   6.108573    4.921118            25.1675
3                8   3.120358    2.429778            26.8981
4               10   1.229338    0.286163            21.2226
5               12   0.848673    0.072301            18.3057
